# Importar Lib's

In [8]:
import os
import openpyxl
import pandas as pd
import ipywidgets as widgets
import numpy as np
from IPython.display import display
from datetime import datetime
from ipywidgets import interact

# Passo 1: Listar arquivos PDF em uma pasta específica

## Pasta contendo os arquivos PDF

In [9]:
diretorio_lote = r'C:\Users\helym\OneDrive - ONE ENGENHARIA INDUSTRIA E COMERCIO LTDA\OeM\OeM\00-INTELBRAS\000_Relatórios de Campo\001 RELATORIOS ENVIADOS\AC\0_0-PREVENTIVA\1ªTRACHE\2024'
arquivos_pdf = []
for raiz, diretorios, arquivos in os.walk(diretorio_lote):
    for arquivo in arquivos:
        if arquivo.endswith('.pdf'):
            arquivos_pdf.append(os.path.join(raiz, arquivo))

duplicadas = pd.DataFrame(arquivos_pdf, columns=['NUMOS'])
duplicadas['NUMOS'] = duplicadas['NUMOS'].apply(lambda x: x[-22:-10])
lista_arquivos_pdf = duplicadas#lista_arquivos_pdf = duplicadas.astype(int)
lista_arquivos_pdf_duplicada = lista_arquivos_pdf.duplicated().sum()
duplicadas.count()

NUMOS    1152
dtype: int64

# Lista de arquivos PDF na pasta

In [10]:
def encontrar_duplicados(duplicadas):
    duplicados = duplicadas[duplicadas.duplicated(subset='NUMOS')]
    return duplicados['NUMOS']

# Exemplo de uso
duplicados = encontrar_duplicados(duplicadas)
print(duplicados)

Series([], Name: NUMOS, dtype: object)


# Passo 2: Ler o arquivo Excel com a lista de arquivos

In [11]:
df_filtered_global_oem = None

def ler_planilha():
    """Lê uma planilha do Excel."""
    excluir_status = ['TREINAMENTO','DUPLICADO','CANCELADO ']
    df_bd = pd.read_excel(r"C:\Users\helym\projetos_python\dashboard\content\base_sip_Concluido.xlsx")
    df = df_bd[~df_bd['STATUS'].isin(excluir_status)]
    return df

def aplicar_filtros(rota_value, status_value, tipo_value, executor_value, origem_value, tipocausa_value, data_inicial, data_final):
    """Aplica filtros a uma planilha."""
    global df_filtered_global_oem
    df_filtered_global_oem = df[
        df["ROTA"].isin(rota_value) &
        df["STATUS"].isin(status_value) &
        df["TIPO"].isin(tipo_value) &
        df["EXECUTOR"].isin(executor_value) &
        df["ORIGEM"].isin(origem_value) &
        df["TIPOCAUSA"].isin(tipocausa_value) &
        (df["DATACONCLUSAO"] >= data_inicial) &
        (df["DATACONCLUSAO"] <= data_final)
    ]
    return df_filtered_global_oem

if __name__ == "__main__":
    df = ler_planilha()

    df["DATACONCLUSAO"] = pd.to_datetime(df["DATACONCLUSAO"]).dt.date

    rota_options = sorted(list(df['ROTA'].unique()))
    rota_filter = widgets.SelectMultiple(options=rota_options, description="Rotas:")

    status_options = sorted(list(df['STATUS'].unique()))
    status_filter = widgets.SelectMultiple(options=status_options, description="Status:")

    tipo_options = sorted(list(df['TIPO'].unique()))
    tipo_filter = widgets.SelectMultiple(options=tipo_options, description="Tipo:")

    executor_options = sorted(list(df['EXECUTOR'].unique()))
    executor_filter = widgets.SelectMultiple(options=executor_options, description="Usuário:")

    origem_options = sorted(list(df['ORIGEM'].unique()))
    origem_filter = widgets.SelectMultiple(options=origem_options, description="Origem:")

    tipocausa_options = sorted(list(df['TIPOCAUSA'].unique()))
    tipocausa_filter = widgets.SelectMultiple(options=tipocausa_options,descrition="Tipo Causa")

    data_inicial_filter = widgets.DatePicker(value=df["DATACONCLUSAO"].min())
    data_final_filter = widgets.DatePicker(value=df["DATACONCLUSAO"].max())

    aplicar_filter_button = widgets.Button(description="Aplicar filtros")

    def on_button_click(event):
        global df_filtered_global_oem
        df_filtrado = aplicar_filtros(
            rota_filter.value, status_filter.value, tipo_filter.value,
            executor_filter.value, origem_filter.value,
            tipocausa_filter.value,
            data_inicial_filter.value, data_final_filter.value
        )
        df_filtered_global_oem = df_filtrado

    aplicar_filter_button.on_click(on_button_click)

    display(data_inicial_filter)
    display(data_final_filter)
    df_filtered_global_oem
    display(
        widgets.HBox([origem_filter, tipo_filter, rota_filter, executor_filter, status_filter],
                     layout=widgets.Layout(align_items='baseline'))
    )
    display(
        widgets.HBox([tipocausa_filter, aplicar_filter_button],
                     layout=widgets.Layout(align_items='center'))
    )

DatePicker(value=datetime.date(2022, 1, 1), step=1)

DatePicker(value=datetime.date(2024, 5, 19), step=1)

In [17]:
caminho_arquivo_excel = df_filtered_global_oem

In [18]:
df_filtro = caminho_arquivo_excel[['NUMOS', 'UC','IDSIGFI','NOMECLIENTE','ROTA','STATUS','EXECUTOR','DTCONCLUSAO']]

## DataFrame a partir do arquivo Excel

In [19]:
df_excel =df_filtro[['NUMOS']]
df_excel.count()

NUMOS    1147
dtype: int64

# Passo 3: Criar um DataFrame com a lista de arquivos PDF

In [20]:
df_pdf = lista_arquivos_pdf.astype('int64')


#  Passo 4: Comparar os DataFrames para identificar arquivos ausentes no PDF e arquivos não listados no Excel

In [21]:
arquivos_ausentes_no_pdf = df_excel[~df_excel['NUMOS'].isin(df_pdf['NUMOS'])]
arquivos_nao_listados_no_excel = df_pdf[~df_pdf['NUMOS'].isin(df_excel['NUMOS'])]

## Resultados

In [22]:
nao_encontado_pdf = arquivos_ausentes_no_pdf.merge(df_filtro, left_on='NUMOS', right_on='NUMOS', how='left')

display(nao_encontado_pdf)
print(nao_encontado_pdf.shape)

,NUMOS,UC,IDSIGFI,NOMECLIENTE,ROTA,STATUS,EXECUTOR,DTCONCLUSAO
0,202400021384,658097.0,14451,MARNILDO CERQUEIRA,8.0,VALIDADO,ONEOM008,2024-05-02 16:18:00
1,202400020154,688884.0,15799,FRANCISCO DAS CHAGAS DE AMORIM SILVA,13.0,VALIDADO,ONEOM007,2024-04-25 10:59:00
2,202400018554,688889.0,15779,FRANCISCO ROSINEI DA SILVA LIMA,13.0,VALIDADO,ONEOM007,2024-04-26 15:52:00
3,202400018547,688894.0,15777,JOSE BERNARDINO DE LIMA,13.0,VALIDADO,ONEOM007,2024-04-26 15:04:00


(4, 8)


In [23]:
nao_encontado_lista = arquivos_nao_listados_no_excel.merge(df_filtro, left_on='NUMOS', right_on='NUMOS', how='left')

display(nao_encontado_lista)
print(nao_encontado_pdf.shape)

,NUMOS,UC,IDSIGFI,NOMECLIENTE,ROTA,STATUS,EXECUTOR,DTCONCLUSAO
0,202400030815,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,202400030831,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,202400030781,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,202400030757,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,202400030773,NaN,NaN,NaN,NaN,NaN,NaN,NaT
5,202400030849,NaN,NaN,NaN,NaN,NaN,NaN,NaT
6,202400030914,NaN,NaN,NaN,NaN,NaN,NaN,NaT
7,202400030906,NaN,NaN,NaN,NaN,NaN,NaN,NaT
8,202400030856,NaN,NaN,NaN,NaN,NaN,NaN,NaT


(4, 8)


In [24]:
print("listar arquivo duplicados:")
display(lista_arquivos_pdf_duplicada)

print("Arquivos ausentes na lista de PDF:")
display(arquivos_ausentes_no_pdf.shape)

print("\nArquivos não listados no Excel:")
display(arquivos_nao_listados_no_excel.shape)

listar arquivo duplicados:


0

Arquivos ausentes na lista de PDF:


(4, 1)


Arquivos não listados no Excel:


(9, 1)

In [ ]:
# Salvando a planilha( .xlsx)
# dataframe_excel = pd.ExcelWriter(r"C:\Users\helym\projetos_python\jupyter\INS\output\1ªtranche_preventiva_pdf_ausente.xlsx")
# nao_encontado_pdf.to_excel(dataframe_excel, index=False)
# dataframe_excel.close()